In [47]:
import pandas as pd
import numpy as np
import aggregation

In [48]:
# open elections_with_tracts.xlsx
elections_with_tracts = pd.read_excel('elections_with_tracts.xlsx')
# open acs_data.csv
acs_data = pd.read_csv('acs_data.csv')
# remove all columns ending in EA
acs_data = acs_data.loc[:, ~acs_data.columns.str.endswith('EA')]
# replace all (X), -, or N with nan
acs_data = acs_data.replace('(X)', np.nan)
acs_data = acs_data.replace('-', np.nan)
acs_data = acs_data.replace('N', np.nan)
# convert all columns except the first 2 to floats
for col in acs_data.columns[2:]:
    acs_data[col] = acs_data[col].astype(float)
    # replace all negative values with NaN except the first 4 columns
    acs_data[col] = acs_data[col].apply(lambda x: np.nan if x < 0 else x)
acs_data

/var/folders/52/t8__xqw525g96s6z30kmvnfr0000gp/T/ipykernel_96675/3225282608.py:4: DtypeWarning: Columns (3,21,23,30,33,48,60,61,64,71,88,134,154,155,168,172,182,205,206,216,218,221,222,246,252,263,264,268,274,277,282,288,291,293,295,304,307,308,314,323,337,350,381,386,415,456,462,488,544,548,557,560,562,567,578,579,581,600,610,611,630,635,645,648,655,664,678,684,699,704,709,710,724,732,736,744,750) have mixed types. Specify dtype option on import or set low_memory=False.
  acs_data = pd.read_csv('acs_data.csv')


,geocode,year,DP02_0127PE,DP02_0039PE,DP02_0130E,DP03_0051E,DP03_0010E,DP03_0121E,DP02_0069PE,DP02_0090E,...,DP03_0115E,DP02_0124PE,DP05_0008PE,DP02_0154PE,DP02_0062E,DP03_0067E,DP05_0003E,DP02_0033PE,DP02_0046PE,DP03_0063E
0,06_001_422200,2010,2.6,NaN,13.0,1522.0,1506.0,NaN,3.3,1044.0,...,318.0,1.3,8.4,NaN,228.0,15020.0,1763.0,2.0,0.0,79904.0
1,06_001_422400,2010,1.6,NaN,0.0,2171.0,1597.0,NaN,1.3,1095.0,...,1425.0,0.3,32.4,NaN,353.0,14688.0,1691.0,0.0,0.0,66278.0
2,06_001_422300,2010,2.4,NaN,10.0,1591.0,1488.0,NaN,5.3,1198.0,...,466.0,1.1,17.9,NaN,392.0,12681.0,1781.0,1.7,0.0,77234.0
3,06_001_420400,2010,0.0,NaN,8.0,1160.0,1238.0,NaN,1.1,487.0,...,965.0,2.1,2.5,NaN,148.0,12903.0,1777.0,5.5,0.0,51303.0
4,06_001_421800,2010,1.4,NaN,0.0,897.0,1025.0,NaN,5.3,784.0,...,213.0,0.0,5.1,NaN,177.0,18863.0,1180.0,0.5,NaN,110333.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6497,02_170_000101,2022,1.4,NaN,28.0,270.0,194.0,NaN,470.0,499.0,...,72.0,512.0,2.7,74.8,200.0,18418.0,213.0,64.4,0.0,77947.0
6498,02_050_000200,2022,0.1,NaN,141.0,1937.0,2058.0,NaN,4403.0,5919.0,...,526.0,6312.0,7.9,91.8,1622.0,18060.0,2878.0,41.9,0.0,118182.0
6499,50_007_003600,2023,0.3,NaN,896.0,2511.0,2624.0,NaN,4448.0,4255.0,...,374.0,4990.0,2.1,79.3,673.0,19693.0,2928.0,25.4,0.0,78025.0
6500,50_007_000800,2023,0.6,NaN,687.0,1032.0,1130.0,NaN,1962.0,2331.0,...,193.0,2430.0,2.8,96.9,180.0,27227.0,1276.0,37.3,0.0,156711.0


In [49]:
# open variables/variables_acs5_2022.csv
variables = pd.read_csv('variables/variables_acs5_2022.csv')

percent_columns = variables[variables['Label'].str.lower().str.contains(r'\brate\b|\bpercentage\b')]
percent_columns['Name'] = percent_columns['Name'].str[:-2]
# for each row in acs_data
for index, row in acs_data.iterrows():
    # for each column in percent_columns
    for col in percent_columns['Name']:
        # if col + E and col + PE are in acs columns
        if col + 'E' in acs_data.columns and col + 'PE' in acs_data.columns:
            # get col + E and col + PE
            est = row[col + 'E']
            percent_est = row[col + 'PE']

            # if percent_est is nan, set PE to est and set E to nan
            if np.isnan(percent_est) or percent_est == 0:
                acs_data.at[index, col + 'PE'] = est
                acs_data.at[index, col + 'E'] = np.nan

/var/folders/52/t8__xqw525g96s6z30kmvnfr0000gp/T/ipykernel_96675/2253298472.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  percent_columns['Name'] = percent_columns['Name'].str[:-2]


In [50]:
# checking for columns with invalid data
invalid = []
# for each column except the first 2
for col in acs_data.columns[2:]:
    valid = False
    # for each value in the column
    for val in acs_data[col]:
        # if the value is not nan and is greater than 0
        if not np.isnan(val) and val > 0:
            valid = True
            break
    if not valid:
        invalid.append(col)
# remove invalid columns
acs_data = acs_data.drop(columns=invalid)
acs_data

,geocode,year,DP02_0127PE,DP02_0130E,DP03_0051E,DP03_0010E,DP02_0069PE,DP02_0090E,DP05_0072PE,DP03_0011PE,...,DP03_0115E,DP02_0124PE,DP05_0008PE,DP02_0154PE,DP02_0062E,DP03_0067E,DP05_0003E,DP02_0033PE,DP02_0046PE,DP03_0063E
0,06_001_422200,2010,2.6,13.0,1522.0,1506.0,3.3,1044.0,60.7,64.3,...,318.0,1.3,8.4,NaN,228.0,15020.0,1763.0,2.0,0.0,79904.0
1,06_001_422400,2010,1.6,0.0,2171.0,1597.0,1.3,1095.0,42.4,60.7,...,1425.0,0.3,32.4,NaN,353.0,14688.0,1691.0,0.0,0.0,66278.0
2,06_001_422300,2010,2.4,10.0,1591.0,1488.0,5.3,1198.0,59.1,71.0,...,466.0,1.1,17.9,NaN,392.0,12681.0,1781.0,1.7,0.0,77234.0
3,06_001_420400,2010,0.0,8.0,1160.0,1238.0,1.1,487.0,29.8,39.2,...,965.0,2.1,2.5,NaN,148.0,12903.0,1777.0,5.5,0.0,51303.0
4,06_001_421800,2010,1.4,0.0,897.0,1025.0,5.3,784.0,73.5,63.9,...,213.0,0.0,5.1,NaN,177.0,18863.0,1180.0,0.5,NaN,110333.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6497,02_170_000101,2022,1.4,28.0,270.0,194.0,470.0,499.0,512.0,47.4,...,72.0,512.0,2.7,74.8,200.0,18418.0,213.0,64.4,0.0,77947.0
6498,02_050_000200,2022,0.1,141.0,1937.0,2058.0,4403.0,5919.0,6312.0,67.7,...,526.0,6312.0,7.9,91.8,1622.0,18060.0,2878.0,41.9,0.0,118182.0
6499,50_007_003600,2023,0.3,896.0,2511.0,2624.0,4448.0,4255.0,4990.0,72.3,...,374.0,4990.0,2.1,79.3,673.0,19693.0,2928.0,25.4,0.0,78025.0
6500,50_007_000800,2023,0.6,687.0,1032.0,1130.0,1962.0,2331.0,2430.0,68.6,...,193.0,2430.0,2.8,96.9,180.0,27227.0,1276.0,37.3,0.0,156711.0


In [51]:
# open statistics.xlsx
statistics = pd.read_excel('statistics.xlsx')
statistics

,agg_var,type,pop_var
0,DP03_0119PE,percentage,DP03_0075E
1,DP03_0120PE,percentage,DP02_0014E
2,DP03_0121PE,percentage,NaN
3,DP03_0122PE,percentage,DP02_0002E
4,DP03_0123PE,percentage,DP02_0003E
5,DP03_0124PE,percentage,NaN
6,DP03_0125PE,percentage,DP02_0010E
7,DP03_0126PE,percentage,DP02_0011E
8,DP03_0127PE,percentage,NaN
9,DP03_0128PE,percentage,DP05_0001E


In [52]:
# for each column in acs_data
for col in acs_data.columns:
   # if it ends in E and not PE
   if col in statistics['agg_var'].values:
      # if pop_var is NaN, delete col from acs_data
      if statistics[statistics['agg_var'] == col]['pop_var'].isnull().values[0]:
         acs_data = acs_data.drop(columns=[col])
      elif col.endswith('PE') and (col[:-2] + 'E') in acs_data.columns:
         # drop the corresponding E column
         acs_data = acs_data.drop(columns=[col[:-2] + 'E'])
   # if it ends in E and PE in columns
   if col.endswith('E') and (col[:-1] + 'PE') not in statistics['agg_var'].values and (col[:-1] + 'PE') in acs_data.columns:
      # drop the corresponding PE column
      acs_data = acs_data.drop(columns=[col[:-1] + 'PE'])

In [53]:
# open elections.xlsx
elections = pd.read_excel('elections.xlsx')
elections

,filename,city,state,Geography,District #,office,year,geo_filename
0,Berkeley_11022010_CityCouncilDistrict1.csv,Berkeley,CA,city council,1,City council,2010,berkeley_city_council_2002.csv
1,Berkeley_11042014_CityCouncilDistrict1.csv,Berkeley,CA,city council,1,City council,2014,berkeley_city_council_2014.csv
2,Berkeley_11062018_CityCouncilDistrict1.csv,Berkeley,CA,city council,1,City council,2018,berkeley_city_council_2014.csv
3,Minneapolis_11022021_CityCouncilWard1.csv,Minneapolis,MN,city council,1,City council,2021,minneapolis_city_council_2012.csv
4,Minneapolis_11052013_Ward13CityCouncil.csv,Minneapolis,MN,city council,1,City council,2013,minneapolis_city_council_2012.csv
...,...,...,...,...,...,...,...,...
494,Cambridge_11062007_SchoolCommittee.csv,Cambridge,MA,school board,NaN,School committee,2007,NaN
495,Cambridge_11052013_SchoolCommittee.csv,Cambridge,MA,school board,NaN,School committee,2013,NaN
496,Cambridge_11072017_SchoolCommittee.csv,Cambridge,MA,school board,NaN,School committee,2017,NaN
497,SanFrancisco_11052019_Sheriff.csv,San Francisco,CA,city,NaN,Sheriff,2019,NaN


In [57]:
# get rows in elections_with_tracts where filename starts with SanFrancisco_11072006
elections_with_tracts = elections_with_tracts[elections_with_tracts['filename'].str.startswith('SanFrancisco_11072006')]
elections_with_tracts

,filename,District #,tracts,percents,num_tracts
41,SanFrancisco_11072006_BoardofSupervisorsDistri...,2,"06_075_012800, 06_075_012700, 06_075_012600, 0...","1.0000000000000002, 0.6867040819702476, 0.5895...",34
105,SanFrancisco_11072006_BoardofSupervisorsDistri...,4,"06_075_060400, 06_075_060300, 06_075_035400, 0...","3.492938156577059e-05, 0.0003440494961896735, ...",19
167,SanFrancisco_11072006_BoardofSupervisorsDistri...,6,"06_041_124200, 06_001_427500, 06_075_012500, 0...","1.9624489541486458e-06, 3.722047252574471e-06,...",31
213,SanFrancisco_11072006_BoardofSupervisorsDistri...,8,"06_075_031100, 06_075_030700, 06_075_030600, 0...","0.18111828539920496, 0.0536714844733811, 2.162...",31
235,SanFrancisco_11072006_BoardofSupervisorsDistri...,10,"06_081_600200, 06_081_600300, 06_075_026302, 0...","5.073828226923904e-05, 0.0009464932948255446, ...",38


In [59]:
all_data = []

# create df called missing_data with columns filename, year, values, num_missing, num_total, missing_tracts
missing_data = pd.DataFrame(columns=['filename', 'year', 'variable', 'values', 'num_missing', 'num_total', 'prop_missing'])

for index, row in elections_with_tracts.iterrows():
    if index % 10 == 0:
        print(f'{index} of {len(elections_with_tracts)} rows processed')

    tracts = row['tracts'].split(',')
    
    # create percents dict 
    percent_dict = {tract: percent for tract, percent in zip(tracts, list(map(float, row['percents'].split(','))))}

    # year is year in row in elections where filename = row['filename']
    year = elections[elections['filename'] == row['filename']]['year'].values[0]
    
    # get rows where geocode is in tracts and year = year
    tract_rows = acs_data[(acs_data['geocode'].isin(tracts)) & (acs_data['year'] == year) & acs_data['DP05_0001E'] > 0]
    # get percents in order of the geocodes in tract_rows
    percents = [percent_dict[geocode] for geocode in tract_rows['geocode']]

    # add percents as column to tract_rows
    tract_rows['percent'] = percents

    election_data = {}

    for column in acs_data.columns[2:]:
        # get the values in tract_rows for column
        values = tract_rows[column].values
        valid_values = values[~np.isnan(values)]

        # if all values are NaN, continue
        if all(np.isnan(values)):
            continue
        else:
        # if at least one but not all values are NaN
         if any(np.isnan(values)):
            # add to missing_data
            missing_data.loc[len(missing_data)] = [row['filename'], year, column, values, sum(np.isnan(values)), len(values), sum(np.isnan(values)) / len(values)]

        if column in statistics['agg_var'].values:
            pop_var = statistics[statistics['agg_var'] == column]['pop_var'].values[0]
            # if pop_var is none skip
            if pd.isnull(pop_var):
                continue

            # if pop_var contains -
            if '-' in pop_var:
                # split on -
                pop_vars = pop_var.split('-')
                pops = tract_rows[pop_vars[0]].values - tract_rows[pop_vars[1]].values
            else:
                pops = tract_rows[pop_var].values
            
            percents = tract_rows['percent'].values
            tract_pops = pops*percents
            valid_tract_pops = tract_pops[~np.isnan(values)]

            if statistics[statistics['agg_var'] == column]['type'].values[0] == 'mean':
                election_data[column] = aggregation.weighted_amean(valid_values, valid_tract_pops)
            elif statistics[statistics['agg_var'] == column]['type'].values[0] in ['percentage', 'ratio']:
                election_data[column] = aggregation.weighted_amean(valid_values / 100, valid_tract_pops) * 100
            elif statistics[statistics['agg_var'] == column]['type'].values[0] == 'median':
                election_data[column] = aggregation.weighted_gmean(valid_values, valid_tract_pops)
        else:
            # if every value is not nan
            if any(~np.isnan(values)):
                pops = tract_rows['DP05_0001E'].values
                percents = tract_rows['percent'].values
                tract_pops = pops*percents
                # get the values in values that are not NaN as well as the corresponding values in DP05_0001E and the corresponding percents
                valid_pops = pops[~np.isnan(values)]
                valid_tract_pops = tract_pops[~np.isnan(values)]

                props = valid_values / valid_pops
                weighted_prop = aggregation.weighted_amean(props, valid_tract_pops)

                # replace the Nan values in values with tract_pop * weighted_prop
                for i in range(len(values)):
                    if np.isnan(values[i]):
                        values[i] = pops[i] * weighted_prop

            election_data[column] = aggregation.weighted_sum(values, percents)

    all_data.append(election_data)

/var/folders/52/t8__xqw525g96s6z30kmvnfr0000gp/T/ipykernel_96675/1809774773.py:13: RuntimeWarning: assigning None to unbound local 'tract'
  percent_dict = {tract: percent for tract, percent in zip(tracts, list(map(float, row['percents'].split(','))))}
/var/folders/52/t8__xqw525g96s6z30kmvnfr0000gp/T/ipykernel_96675/1809774773.py:13: RuntimeWarning: assigning None to unbound local 'percent'
  percent_dict = {tract: percent for tract, percent in zip(tracts, list(map(float, row['percents'].split(','))))}
/var/folders/52/t8__xqw525g96s6z30kmvnfr0000gp/T/ipykernel_96675/1809774773.py:13: RuntimeWarning: assigning None to unbound local 'tract'
  percent_dict = {tract: percent for tract, percent in zip(tracts, list(map(float, row['percents'].split(','))))}
/var/folders/52/t8__xqw525g96s6z30kmvnfr0000gp/T/ipykernel_96675/1809774773.py:13: RuntimeWarning: assigning None to unbound local 'percent'
  percent_dict = {tract: percent for tract, percent in zip(tracts, list(map(float, row['percents']

KeyboardInterrupt: 

In [40]:
# save missing_data to csv
missing_data.to_csv('missing_data.csv', index=False)

In [56]:
all_data_df = pd.DataFrame(all_data)
result_df = pd.concat([elections_with_tracts, all_data_df], axis=1)
result_df

,filename,District #,tracts,percents,num_tracts
41,SanFrancisco_11072006_BoardofSupervisorsDistri...,2,"06_075_012800, 06_075_012700, 06_075_012600, 0...","1.0000000000000002, 0.6867040819702476, 0.5895...",34.0
105,SanFrancisco_11072006_BoardofSupervisorsDistri...,4,"06_075_060400, 06_075_060300, 06_075_035400, 0...","3.492938156577059e-05, 0.0003440494961896735, ...",19.0
167,SanFrancisco_11072006_BoardofSupervisorsDistri...,6,"06_041_124200, 06_001_427500, 06_075_012500, 0...","1.9624489541486458e-06, 3.722047252574471e-06,...",31.0
213,SanFrancisco_11072006_BoardofSupervisorsDistri...,8,"06_075_031100, 06_075_030700, 06_075_030600, 0...","0.18111828539920496, 0.0536714844733811, 2.162...",31.0
235,SanFrancisco_11072006_BoardofSupervisorsDistri...,10,"06_081_600200, 06_081_600300, 06_075_026302, 0...","5.073828226923904e-05, 0.0009464932948255446, ...",38.0
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN


In [42]:
# save result_df as final_data.csv
result_df.to_csv('final_data.csv', index=False)